In [ ]:
import pandas as pd
pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", None)
import tkinter as tk
from tkinter import filedialog

In [ ]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
df = pd.read_excel(file_path, sheet_name="出貨明細")

In [ ]:
iris = df[["BG", "Subcategory", "Group", "狀態", "預交年份", "預交月份", "負責業務", "品名", "本國幣別NTD"]]

In [ ]:
iris = iris[(iris["BG"] == "RF") & (iris["狀態"].str.contains("出")) & (iris["預交年份"] == 2021) & (iris["預交月份"].isin(["February", "March", "April"]))]

In [ ]:
iris["Subcategory"] = iris.apply(lambda x: "BU2" if "RFDPA" in x["品名"] else "BU1", axis=1)

In [ ]:
iris = iris[iris["Subcategory"] == "BU1"]

In [ ]:
budget = pd.read_excel(file_path, sheet_name="2021預算總表", usecols="A:I")

In [ ]:
revenue = iris.groupby(["負責業務", "預交月份"])["本國幣別NTD"].sum().reset_index()

In [ ]:
budget = budget[(budget["類型"] == "天線") & (budget["預交年份"] == 2021) & (budget["預交月份"].isin(["February", "March", "April"]))].groupby(["負責業務", "預交月份"])[["本國幣別NTD"]].sum().reset_index()

In [ ]:
budget["類別"] = "預算"
revenue["類別"] = "實績"

In [ ]:
budget.columns = ['負責業務', '預交月份', '本國幣別NTD', '類別']

In [ ]:
revenue.columns = ['負責業務', '預交月份', '本國幣別NTD', '類別']

In [ ]:
for_iris = pd.concat([revenue, budget], axis=0)

In [ ]:
result = iris.pivot_table(index=["負責業務", "Subcategory"], values="本國幣別NTD", columns="預交月份", aggfunc="sum").reset_index()

In [ ]:
'''.xs()可以將unstack後的level刪除'''
test = iris.groupby(["負責業務", "Subcategory", "預交月份"])[["本國幣別NTD"]].sum().unstack("預交月份").xs("本國幣別NTD", axis=1).reset_index()

In [ ]:
result = iris.pivot_table(index=["負責業務", "Subcategory"], columns="預交月份", values="本國幣別NTD", aggfunc="sum", margins=True, margins_name="Total").reset_index()

In [ ]:
result = for_iris.pivot_table(index=["負責業務", "類別"], columns="預交月份", values="本國幣別NTD", aggfunc="sum").reset_index()

In [ ]:
final = pd.DataFrame(result.to_records())

In [ ]:
final = final.drop("index", axis=1)

In [ ]:
final

In [ ]:
final = final.reindex(columns=['負責業務', '類別', 'February', 'March', 'April'])

In [ ]:
# final = final.sort_values(["Subcategory", "February", "March"], ascending=[True, False, False])
final.style.format({"February":"{0:,.0f}", "March":"{0:,.0f}", "April":"{:,.0f}"}).hide_index()

In [ ]:
result.style.format({"本國幣別NTD":"{0:,.0f}"}).highlight_max(color="lightgreen").highlight_min(color="red")